In [1]:
import json
import re

from ollama import Client

client = Client(host="http://127.0.0.1:11434")
with open("sdg_label_prompt.md", "r") as f:
    prompt_template = f.read()
        


In [2]:
def get_sdg_reason(text):

    reason_match = re.search(r'<reason>(.*?)</reason>', text, re.DOTALL)
    sdg_match = re.search(r'<sdg>(.*?)</sdg>', text, re.DOTALL)

    reason_content_regex = ""
    sdg_content_regex = ""

    if reason_match:
        reason_content_regex = reason_match.group(1).strip()

    if sdg_match:
        sdg_content_regex = sdg_match.group(1).strip()
    return sdg_content_regex, reason_content_regex

In [ ]:

# Ouvrir le fichier en mode écriture ('w')
# Il est bon d'utiliser encoding='utf-8' pour les fichiers JSON.
with open("../src/ai/testsets/dataset_labeled.jsonl", "w", encoding="utf-8") as f_out:
    with open("../src/ai/testsets/dataset.jsonl", "r", encoding="utf-8") as f:
        for line in f:
            item = json.loads(line)
            try:
                for key, value in item.items():
                    formatted_prompt = prompt_template.replace("{description}", value)
                    response_data = client.generate(
                        model="qwen3:14b", # Make sure this model is available in your Ollama instance
                        prompt=formatted_prompt,
                        options={"temperature": 0.2}
                    )
                    raw_output_text = response_data.get('response', '').strip()
                    break
                sdg, reason = get_sdg_reason(raw_output_text)
                output_data = {key: sdg, "reason": reason}
                f_out.write(json.dumps(output_data, ensure_ascii=False) + "\n")
                f_out.flush()

            except:
                print(f"Erreur : Pour {key}, {value} n'a pas été trouvé.")
